In [14]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import albumentations as A
from pathlib import Path
import os
from tqdm import tqdm
import shutil

IMG_HEIGHT = 137;
IMG_WIDTH = 236;

counts = dict({'grapheme':168, 'vowels':11, 'consonants':7})
classes_grapheme = range(168)
classes_vowel = range(11)
class_consonants = range(7)

In [15]:
counts['grapheme']

168

In [16]:
datadir = '/home/sayan/Documents/Bengali_Grapheme/data/'
imagedir = '/home/sayan/Documents/Bengali_Grapheme/images/'
train_df = pd.read_csv(datadir+'train.csv')

In [17]:
class_map = pd.read_csv(datadir+'class_map.csv')

In [18]:
count_grapheme_roots = np.unique(train_df['grapheme_root'])
count_vowel_diacritic = np.unique(train_df['vowel_diacritic'])
count_consonant_diacritic = np.unique(train_df['consonant_diacritic'])
print('Unique Grapheme Roots: '+str(len(count_grapheme_roots)))
print('Unique Vowel Diacritcs: '+str(len(count_vowel_diacritic)))
print('Unique Consonant Diacritcs: '+str(len(count_consonant_diacritic)))

Unique Grapheme Roots: 168
Unique Vowel Diacritcs: 11
Unique Consonant Diacritcs: 7


In [19]:
from sklearn.preprocessing import OneHotEncoder

ohe_labels = OneHotEncoder(sparse=False)

y_labels = ohe_labels.fit_transform(train_df[['grapheme_root','vowel_diacritic','consonant_diacritic']],)
label_names = ohe_labels.get_feature_names(['grapheme_root','vowel_diacritic','consonant_diacritic'])
y = pd.DataFrame(data=y_labels,columns = label_names)

/home/sayan/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [20]:
train_df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


In [21]:
v = train_df[train_df['grapheme_root'] == 15]['image_id'].values

In [22]:
# Create training, validation, and testing datasets
base_training_grapheme = "Toy_Training/grapheme"
base_validation_grapheme = "Toy_Validation/grapheme"
base_testing_grapheme = "Toy_Testing/grapheme"

base_training_vowel = "Toy_Training/vowel"
base_validation_vowel = "Toy_Validation/vowel"
base_testing_vowel = "Toy_Testing/vowel"

base_training_consonant = "Toy_Training/consonant"
base_validation_consonant = "Toy_Validation/consonant"
base_testing_consonant = "Toy_Testing/consonant"

folders = ["Toy_Training/grapheme","Toy_Validation/grapheme","Toy_Testing/grapheme",
           "Toy_Training/vowel","Toy_Validation/vowel", "Toy_Testing/vowel",
           "Toy_Training/consonant","Toy_Validation/consonant","Toy_Testing/consonant"]

In [33]:
def create_data_directory(target,dirname,df):
    for label in range(counts[target]):
        class_folder = dirname+'/'+str(label)
        Path(class_folder).mkdir(parents = True)
        if target == 'vowels':
            target_key = 'vowel_diacritic'
        elif target == 'consonants':
            target_key = 'consonant_diacritic'
        else:
            target_key = 'grapheme_root'
        images = (df[df[target_key] == label]['image_id'])
        count = 0
        for image in images:
            shutil.copy(imagedir+'toy_processed_images/'+image+'_f.png', class_folder+'/'+str(count)+'.png')
            count = count+1

In [34]:
datadir_raw_images = Path('/home/sayan/Documents/Bengali_Grapheme/images/toy_processed_images/')
images = []
for (directory, _ , image_names) in os.walk(datadir_raw_images):
        for image_name in tqdm(image_names):
            images.append(image_name.replace('_f.png',''))

toy_train = train_df[train_df['image_id'].isin(images)]

100%|██████████| 20084/20084 [00:00<00:00, 1565449.47it/s]


In [36]:
create_data_directory('vowels','/home/sayan/Documents/Bengali_Grapheme/images/train_vowels',toy_train)

In [37]:
create_data_directory('consonants','/home/sayan/Documents/Bengali_Grapheme/images/train_consonants',toy_train)

In [39]:
create_data_directory('grapheme','/home/sayan/Documents/Bengali_Grapheme/images/train_grapheme',toy_train)